In [0]:
from urllib import request
import pandas as pd
from bs4 import BeautifulSoup



def seisan(*args):
    """
    東京証券取引所から最新の清算値データを取得、抽出する
    :param args: DataFrame のクエリー文字列 
    :return: DataFrame型の清算値データ
    
    :Examples
    --------
    #最新の一月限のコール清算値抽出（権利価格18000 <= STRIKE <= 21000）
    >>>df = seisan(
    "MATURITY in [201901]" ,
    "18000 <= STRIKE <= 21000",
    "TYPE in ['CAL']"
    ) 
    """

    
    sv = "https://www.jpx.co.jp/"
    url = request.urlopen(sv + "markets/derivatives/jnet-derivative/index.html")
    soup = BeautifulSoup(url, "html.parser")
    csv_path = soup.find(class_="component-file").a.get("href")

    print('清算値@', csv_path[-12:-4])
    print('  << ', sv + csv_path)
    print()

    colName = ("CODE", "NAME", "TYPE", "MATURITY", "STRIKE",
               "PRICE", "TPRICE", "NK", "IV", "R", "TIME", "BASE")

    df = pd.read_csv(sv + csv_path, skiprows=3, encoding="SHIFT-JIS", names=colName) 
    df = df.query('BASE=="日経225"')
    for s in args: df = df.query(s)

    return df[["TYPE", "STRIKE", "PRICE", "MATURITY","IV","NAME"]]

In [2]:
df = seisan(
    "MATURITY in [201901]" ,
    "20000 <= STRIKE <= 21000",
   #"TYPE in ['CAL']"
    )

df = df.loc[:,['PRICE']].astype(int).set_index(
    df['TYPE'].str[0] + df['STRIKE'].astype(int).astype(str))

清算値@ 20190109
  <<  https://www.jpx.co.jp//markets/derivatives/jnet-derivative/tvdivq00000014l6-att/rb20190109.csv



In [3]:
df

,PRICE
C20000,475
C20125,370
C20250,270
C20375,180
C20500,105
C20625,60
C20750,32
C20875,15
C21000,7
P20000,27


In [0]:
df.to_csv('data20190109.csv')